In [16]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [17]:
def load_and_prep_cifar(batch_size, shuffle_buffer_size):
    train, test = tfds.load('cifar10', split=['train', 'test'], as_supervised=True)

    def preprocessing_func(img, label):
        img = tf.cast(img, tf.float32)
        img = (img / 128) - 1
        label = tf.one_hot(label, depth=10)
        return img, label

    train = train.map(lambda img, label: preprocessing_func(img, label))
    test = test.map(lambda img, label: preprocessing_func(img, label))

    train = train.shuffle(shuffle_buffer_size).batch(batch_size).prefetch(2)
    test = test.batch(batch_size).prefetch(2)

    return train, test


In [18]:
# Set hyperparameters
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1000
LR = 0.001
NUM_EPOCHS = 50

In [19]:
# Load and preprocess the data
train_data, test_data = load_and_prep_cifar(batch_size=BATCH_SIZE, shuffle_buffer_size=SHUFFLE_BUFFER_SIZE)

# # Visualize a sample from the training dataset
# sample_batch = next(iter(train_data))
# images, labels = sample_batch

# plt.figure(figsize=(10, 10))
# for i in range(9):
#     plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy() / 2 + 0.5)  # Rescale images to [0, 1]
#     plt.title(f"Label: {tf.argmax(labels[i]).numpy()}")
#     plt.axis("off")
# plt.show()

In [20]:
def train_cifar_10(model, train_ds, test_ds, num_epochs, lr):
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")

    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []

    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def test_step(images, labels):
        predictions = model(images)
        t_loss = loss_object(labels, predictions)

        test_loss(t_loss)
        test_accuracy(labels, predictions)

    for epoch in range(num_epochs):
        # Training loop
        for images, labels in train_ds:
            train_step(images, labels)

        # Testing loop
        for test_images, test_labels in test_ds:
            test_step(test_images, test_labels)

        # Save metrics for plotting
        train_losses.append(train_loss.result().numpy())
        train_accuracies.append(train_accuracy.result().numpy())
        test_losses.append(test_loss.result().numpy())
        test_accuracies.append(test_accuracy.result().numpy())

        template = 'Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}%, Test Loss: {:.4f}, Test Accuracy: {:.2f}%'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result() * 100,
                              test_loss.result(),
                              test_accuracy.result() * 100))

        # Reset the metrics for the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

    # Plotting
    plt.figure(figsize=(12, 4))

    # Plot training loss
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train')
    plt.plot(test_losses, label='Test')
    plt.title('Training and Testing Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train')
    plt.plot(test_accuracies, label='Test')
    plt.title('Training and Testing Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [25]:
# Define and compile the model
def create_my_cifar_cnn(name='my_cnn'):
    inputs = tf.keras.Input(shape=(32, 32, 3), dtype=tf.float32)

    cnn_layer_1_1= tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_1_1(inputs)#shape: [batch_size, 32,32,16]

    cnn_layer_1_2= tf.keras.layers.Conv2D(filters = 16, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_1_2(x)#shape: [batch_size, 32,32,16]

    cnn_layer_pooling1 = tf.keras.layers.MaxPool2D()
    x= cnn_layer_pooling1(x)#shape: [batch_size, 16,16,16]

    cnn_layer_2_1= tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_2_1(x)#shape: [batch_size, 16,16,32]

    cnn_layer_2_2= tf.keras.layers.Conv2D(filters = 32, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_2_2(x)#shape: [batch_size, 16,16,32]

    cnn_layer_pooling2 = tf.keras.layers.MaxPool2D()
    x= cnn_layer_pooling2(x)#shape: [batch_size, 8,8,32]

    cnn_layer_3_1= tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_3_1(x)#shape: [batch_size, 8,8,64]

    cnn_layer_3_2= tf.keras.layers.Conv2D(filters = 64, kernel_size=(3,3), activation = 'relu',padding='same')
    x= cnn_layer_3_2(x)#shape: [batch_size, 8,8,64]

    cnn_layer_flatten = tf.keras.layers.Flatten()
    x= cnn_layer_flatten(x)# shape: [batch_size, 4096]

    dense_layer_1 = tf.keras.layers.Dense(64, activation = 'relu')
    x= dense_layer_1(x)#shape: [batch_size, 64]

    dense_layer_2 = tf.keras.layers.Dense(32, activation = 'relu')
    x= dense_layer_2(x)#shape: [batch_size, 32]

    output_layer = tf.keras.layers.Dense(10, activation = 'softmax')
    output = output_layer(x)#shape: [batch_size, 10]

    model = tf.keras.Model(inputs , output,name=name)

    return model

In [ ]:
model = create_my_cifar_cnn()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model using the custom training loop
train_cifar_10(model, train_data, test_data, num_epochs=NUM_EPOCHS, lr=LR)

Epoch 1, Loss: 1.5423, Accuracy: 43.35%, Test Loss: 1.2651, Test Accuracy: 54.33%
Epoch 2, Loss: 1.0943, Accuracy: 60.83%, Test Loss: 1.0594, Test Accuracy: 62.53%
Epoch 3, Loss: 0.9035, Accuracy: 67.92%, Test Loss: 0.8775, Test Accuracy: 68.86%
Epoch 4, Loss: 0.7775, Accuracy: 72.59%, Test Loss: 0.8223, Test Accuracy: 71.48%
Epoch 5, Loss: 0.6734, Accuracy: 76.22%, Test Loss: 0.8192, Test Accuracy: 72.56%
Epoch 6, Loss: 0.5935, Accuracy: 79.23%, Test Loss: 0.8205, Test Accuracy: 72.97%
Epoch 7, Loss: 0.5139, Accuracy: 82.00%, Test Loss: 0.8761, Test Accuracy: 72.74%
Epoch 8, Loss: 0.4451, Accuracy: 84.45%, Test Loss: 0.9162, Test Accuracy: 73.58%
Epoch 9, Loss: 0.3808, Accuracy: 86.77%, Test Loss: 0.9701, Test Accuracy: 72.85%
Epoch 10, Loss: 0.3248, Accuracy: 88.62%, Test Loss: 1.0316, Test Accuracy: 73.82%
Epoch 11, Loss: 0.2876, Accuracy: 89.84%, Test Loss: 1.1229, Test Accuracy: 72.49%
Epoch 12, Loss: 0.2494, Accuracy: 91.19%, Test Loss: 1.2474, Test Accuracy: 72.57%
Epoch 13, Los